# A Walk Through Ensemble Models
*Complete and hand in this completed worksheet (including its outputs and any supporting code outside of the worksheet) with your assignment submission. Please check the pdf file for more details.*

In this exercise you will:

- get to know a useful package **pandas** for data analysis/preprocessing
- implement **decision tree** and apply it to a Titanic dataset
- implement a whole bunch of **ensemble methods**, including **random forest, and adaboost**, and apply them to a Titanic dataset

Please note that **YOU CANNOT USE ANY MACHINE LEARNING PACKAGE SUCH AS SKLEARN** for any homework, unless you are asked to.

In [1]:
# some basic imports
from scipy import io
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import re

%matplotlib inline

%load_ext autoreload
%autoreload 2

## Let's first do some data preprocessing

Here we use [pandas](https://pandas.pydata.org/) to do data preprocessing. Pandas is a very popular and handy package for data science or machine learning. You can also refer to this official guide for pandas: [10 minutes to pandas](https://pandas.pydata.org/pandas-docs/stable/getting_started/10min.html)

In [2]:
# read titanic train and test data
train = pd.read_csv('./input/train.csv')
test = pd.read_csv('./input/test.csv')

print("train shape: {} test shape: {}".format(train.shape, test.shape))
# Showing overview of the train dataset
train.head(3)

train shape: (1047, 11) test shape: (262, 11)


,Pclass,Survived,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,1,"Hays, Miss. Margaret Bechstein",female,24.0,0,0,11767,83.1583,C54,C
1,3,0,"Holm, Mr. John Fredrik Alexander",male,43.0,0,0,C 7075,6.4500,NaN,S
2,3,0,"Hansen, Mr. Claus Peter",male,41.0,2,0,350026,14.1083,NaN,S


## deal with missing values and transform to discrete variables

In [3]:
# copied from: https://www.kaggle.com/dmilla/introduction-to-decision-trees-titanic-dataset
full_data = [train, test]

# Feature that tells whether a passenger had a cabin on the Titanic
train['Has_Cabin'] = train["Cabin"].apply(lambda x: 0 if type(x) == float else 1)
test['Has_Cabin'] = test["Cabin"].apply(lambda x: 0 if type(x) == float else 1)

# Create new feature FamilySize as a combination of SibSp and Parch
for dataset in full_data:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1
# Create new feature IsAlone from FamilySize
for dataset in full_data:
    dataset['IsAlone'] = 0
    dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 1
# Remove all NULLS in the Embarked column
for dataset in full_data:
    dataset['Embarked'] = dataset['Embarked'].fillna('S')
# Remove all NULLS in the Fare column
for dataset in full_data:
    dataset['Fare'] = dataset['Fare'].fillna(train['Fare'].median())

# Remove all NULLS in the Age column
for dataset in full_data:
    age_avg = dataset['Age'].mean()
    age_std = dataset['Age'].std()
    age_null_count = dataset['Age'].isnull().sum()
    age_null_random_list = np.random.randint(age_avg - age_std, age_avg + age_std, size=age_null_count)
    # Next line has been improved to avoid warning
    dataset.loc[np.isnan(dataset['Age']), 'Age'] = age_null_random_list
    dataset['Age'] = dataset['Age'].astype(int)

# Define function to extract titles from passenger names
def get_title(name):
    title_search = re.search(' ([A-Za-z]+)\.', name)
    # If the title exists, extract and return it.
    if title_search:
        return title_search.group(1)
    return ""

for dataset in full_data:
    dataset['Title'] = dataset['Name'].apply(get_title)
# Group all non-common titles into one single grouping "Rare"
for dataset in full_data:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')

for dataset in full_data:
    # Mapping Sex
    dataset['Sex'] = dataset['Sex'].map( {'female': 0, 'male': 1} ).astype(int)
    
    # Mapping titles
    title_mapping = {"Mr": 1, "Master": 2, "Mrs": 3, "Miss": 4, "Rare": 5}
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)

    # Mapping Embarked
    dataset['Embarked'] = dataset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)
    
    # Mapping Fare
    dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare'] = 2
    dataset.loc[ dataset['Fare'] > 31, 'Fare'] = 3
    dataset['Fare'] = dataset['Fare'].astype(int)
    
    # Mapping Age
    dataset.loc[ dataset['Age'] <= 16, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
    dataset.loc[ dataset['Age'] > 64, 'Age'] = 4

In [4]:
drop_elements = ['Name', 'Ticket', 'Cabin', 'SibSp']
train = train.drop(drop_elements, axis = 1)
test  = test.drop(drop_elements, axis = 1)

In [5]:
train.head()

,Pclass,Survived,Sex,Age,Parch,Fare,Embarked,Has_Cabin,FamilySize,IsAlone,Title
0,1,1,0,1,0,3,1,1,1,1,4
1,3,0,1,2,0,0,0,0,1,1,1
2,3,0,1,2,0,1,0,0,3,0,1
3,3,0,1,0,0,0,2,0,1,1,1
4,2,0,1,2,0,1,0,0,1,1,1


One of the good thing of pd.DataFrame is that you can keep the column names along with the data, which can be beneficial for many case.

Another good thing is that pd.DataFrame can be converted to np.array implicitely.

Also, pd provides a lot of useful data manipulating methods for your convenience, though we may not use them in this homework.

In [6]:
X = train.drop(['Survived'], axis=1)
y = train["Survived"]
X_test = test.drop(['Survived'], axis=1)
y_test = test["Survived"]
print("train: {}, test: {}".format(X.shape, X_test.shape))

train: (1047, 10), test: (262, 10)


In [7]:
def accuracy(y_gt, y_pred):
    return np.sum(y_gt == y_pred) / y_gt.shape[0]

In [8]:
print("Survived: {:.4f}, Not Survivied: {:.4f}".format(y.sum() / len(y), 1 - y.sum() / len(y)))

Survived: 0.3878, Not Survivied: 0.6122


In [9]:
print (X[0:1])

   Pclass  Sex  Age  Parch  Fare  Embarked  Has_Cabin  FamilySize  IsAlone  \
0       1    0    1      0     3         1          1           1        1   

   Title  
0      4  


## Decision Tree
Now it's your turn to do some real coding. Please implement the decision tree model in **decision_tree.py**. The PDF file provides some hints for this part.

In [151]:
def calc_height(d):
    now = 0
    if type(d) == dict:
        for t in list(d.values())[0].values():
            now = max(now, calc_height(t))
    return now + 1

from decision_tree import DecisionTree

# Plot the decision tree to get an intuition about how it makes decision
#plt.figure(figsize=(10, 5))
#dt.show()

dt = DecisionTree(criterion='entropy', max_depth=10, min_samples_leaf=1, sample_feature=False)
dt.fit(X, y)
y_train_pred = dt.predict(X)
print (calc_height(dt._tree))
print("Accuracy on train set: {}".format(accuracy(y, dt.predict(X))))
print("Accuracy on test set: {}".format(accuracy(y_test, dt.predict(X_test))))

10
Accuracy on train set: 0.8787010506208214
Accuracy on test set: 0.7900763358778626


In [104]:
# TODO: Train the best DecisionTree(best val accuracy) that you can. You should choose some 
# hyper-parameters such as critertion, max_depth, and min_samples_in_leaf 
# according to the cross-validation result.
# To reduce difficulty, you can use KFold here.
from sklearn.model_selection import KFold
kf = KFold(n_splits=5, shuffle=True, random_state=2020)

best_acc, best_min_samples_leaf, best_max_depth = 0, 0, 0
best_method = ""
for now_method in ['entropy', 'infogain_ratio', 'gini']:
    for now_min_samples_leaf in range(1, 10):
        for now_max_depth in range(5, 11):
            dt = DecisionTree(criterion=now_method, max_depth=now_max_depth, min_samples_leaf=now_min_samples_leaf, sample_feature=False)
            ave_acc = 0
            for train_indice, valid_indice in kf.split(X, y):
                X_train_fold, y_train_fold = X.loc[train_indice], y.loc[train_indice]
                X_val_fold, y_val_fold = X.loc[valid_indice], y.loc[valid_indice]
                dt.fit(X_train_fold, y_train_fold)
                y_train_pred = dt.predict(X_train_fold) 
                y_valid_pred = dt.predict(X_val_fold)
                ave_acc += accuracy(y_val_fold, y_valid_pred) * 0.7 + accuracy(y_train_fold, y_train_pred) * 0.3
            ave_acc /= 5
            print(now_method, now_min_samples_leaf, now_max_depth, ave_acc)
            if ave_acc > best_acc:
                best_acc = ave_acc
                best_method = now_method
                best_min_samples_leaf = now_min_samples_leaf
                best_max_depth = now_max_depth
    
    
# begin answer
print (best_acc, best_method, best_min_samples_leaf, best_max_depth)
# end answer

entropy 1 5 0.7992937733392289
entropy 1 6 0.799393209786271
entropy 1 7 0.7983865402979309
entropy 1 8 0.8013493436089073
entropy 1 9 0.7966667279789232
entropy 1 10 0.7993397742309168
entropy 2 5 0.8006302964652257
entropy 2 6 0.8014027791642615
entropy 2 7 0.8017326328019181
entropy 2 8 0.7993429640235803
entropy 2 9 0.7993365844382534
entropy 2 10 0.7986762973569136
entropy 3 5 0.7988859410708992
entropy 3 6 0.8039949801715425
entropy 3 7 0.7995988462654102
entropy 3 8 0.7977485094838969
entropy 3 9 0.7990754632319031
entropy 3 10 0.8004119863579
entropy 4 5 0.7996558803345193
entropy 4 6 0.8042667448254687
entropy 4 7 0.8024004590806377
entropy 4 8 0.8045565018844514
entropy 4 9 0.8052263583437815
entropy 4 10 0.8018738862544673
entropy 5 5 0.7980019528240249
entropy 5 6 0.8051488739766676
entropy 5 7 0.8045474267700199
entropy 5 8 0.802619025815366
entropy 5 9 0.8026158360227026
entropy 5 10 0.8046190258153659
entropy 6 5 0.7974416630045076
entropy 6 6 0.8045042259411502
entropy 

In [134]:
# report the accuracy on test set
dt = DecisionTree(criterion=best_method, max_depth=best_max_depth, min_samples_leaf=best_min_samples_leaf, sample_feature=False)
dt.fit(X, y)
print("Accuracy on train set: {}".format(accuracy(y, dt.predict(X))))
print("Accuracy on test set: {}".format(accuracy(y_test, dt.predict(X_test))))

Accuracy on train set: 0.833810888252149
Accuracy on test set: 0.8091603053435115


In [ ]:
dt = DecisionTree(criterion='infogain_ratio', max_depth=8, min_samples_leaf=1, sample_feature=False)

## Random Forest
Please implement the random forest model in **random_forest.py**. The PDF file provides some hints for this part.

In [129]:
from random_forest import RandomForest

base_learner = DecisionTree(criterion='entropy', max_depth=5, min_samples_leaf=5, sample_feature=True)
rf = RandomForest(base_learner=base_learner, n_estimator=100, seed=2020)
rf.fit(X, y)

y_train_pred = rf.predict(X)

print("Accuracy on train set: {}".format(accuracy(y, y_train_pred)))
print("Accuracy on test set: {}".format(accuracy(y_test, rf.predict(X_test))))

Accuracy on train set: 0.8404966571155683
Accuracy on test set: 0.816793893129771


In [165]:
from random_forest import RandomForest

base_learner = DecisionTree(criterion='entropy', max_depth=8, min_samples_leaf=5, sample_feature=True)
rf = RandomForest(base_learner=base_learner, n_estimator=100, seed=2020)
rf.fit(X, y)

y_train_pred = rf.predict(X)

print("Accuracy on train set: {}".format(accuracy(y, y_train_pred)))
print("Accuracy on test set: {}".format(accuracy(y_test, rf.predict(X_test))))

Accuracy on train set: 0.8691499522445081
Accuracy on test set: 0.8015267175572519


In [194]:
# TODO: Train the best RandomForest that you can. You should choose some 
# hyper-parameters such as max_depth, and min_samples_in_leaf 
# according to the cross-validation result.

# For k = 10

from sklearn.model_selection import KFold
kf = KFold(n_splits=5, shuffle=True, random_state=2020)

best_acc, best_min_samples_leaf, best_max_depth = 0, 0, 0
best_method = ""
for now_method in ['infogain_ratio', 'gini', 'entropy']:
    for now_min_samples_leaf in range(1, 10, 1):
        for now_max_depth in range(3, 10, 1):
            base_learner = DecisionTree(criterion=now_method, max_depth=now_max_depth, min_samples_leaf=now_min_samples_leaf, sample_feature=True)
            rf = RandomForest(base_learner=base_learner, n_estimator=10, seed=2020)
            ave_valid_acc, ave_train_acc, ave_com_acc = 0, 0, 0
            for train_indice, valid_indice in kf.split(X, y):
                X_train_fold, y_train_fold = X.loc[train_indice], y.loc[train_indice]
                X_val_fold, y_val_fold = X.loc[valid_indice], y.loc[valid_indice]
                rf.fit(X_train_fold, y_train_fold)
                y_train_pred = rf.predict(X_train_fold) 
                y_valid_pred = rf.predict(X_val_fold)
                train_acc = accuracy(y_train_fold, y_train_pred)
                valid_acc = accuracy(y_val_fold, y_valid_pred)
                ave_train_acc += train_acc
                ave_valid_acc += valid_acc
                ave_com_acc += valid_acc * 0.7 + train_acc * 0.3
                
            ave_train_acc /= 5
            ave_valid_acc /= 5
            ave_com_acc /= 5
            print("%16s %d %d %.4f%% %.4f%% %.4f%%" % (now_method, now_min_samples_leaf, now_max_depth, ave_com_acc*100, ave_train_acc*100, ave_valid_acc*100))
            if ave_com_acc > best_acc:
                best_acc = ave_com_acc
                best_method = now_method
                best_min_samples_leaf = now_min_samples_leaf
                best_max_depth = now_max_depth
    
    
# begin answer
print (best_acc, best_method, best_min_samples_leaf, best_max_depth)
# end answer

  infogain_ratio 1 3 78.1847% 79.2027% 77.7485%
  infogain_ratio 1 4 79.9695% 80.9213% 79.5616%
  infogain_ratio 1 5 80.8908% 82.2112% 80.3249%
  infogain_ratio 1 6 80.9308% 83.2377% 79.9421%
  infogain_ratio 1 7 80.1297% 84.3599% 78.3167%
  infogain_ratio 1 8 80.3261% 86.1270% 77.8401%
  infogain_ratio 1 9 80.4341% 87.8223% 77.2677%
  infogain_ratio 2 3 78.1181% 79.2027% 77.6532%
  infogain_ratio 2 4 79.9695% 80.9213% 79.5616%
  infogain_ratio 2 5 80.8908% 82.2112% 80.3249%
  infogain_ratio 2 6 80.9305% 83.2377% 79.9417%
  infogain_ratio 2 7 79.8617% 84.3599% 77.9339%
  infogain_ratio 2 8 80.3931% 86.1270% 77.9357%
  infogain_ratio 2 9 80.8351% 87.8223% 77.8405%
  infogain_ratio 3 3 78.2325% 78.9163% 77.9394%
  infogain_ratio 3 4 79.6851% 81.0885% 79.0836%
  infogain_ratio 3 5 81.0957% 82.4498% 80.5154%
  infogain_ratio 3 6 80.1519% 83.0944% 78.8909%
  infogain_ratio 3 7 81.1818% 84.0733% 79.9426%
  infogain_ratio 3 8 80.4913% 86.0075% 78.1271%
  infogain_ratio 3 9 80.0777% 87.0819% 7

         entropy 7 6 81.8839% 85.5299% 80.3213%
         entropy 7 7 81.6172% 86.4135% 79.5616%
         entropy 7 8 81.5782% 86.5090% 79.4650%
         entropy 7 9 80.7308% 86.5806% 78.2237%
         entropy 8 3 80.5746% 81.8292% 80.0369%
         entropy 8 4 82.2400% 83.1423% 81.8533%
         entropy 8 5 81.1800% 84.2885% 79.8478%
         entropy 8 6 81.1433% 85.5060% 79.2736%
         entropy 8 7 81.0764% 86.1748% 78.8913%
         entropy 8 8 81.3948% 86.3420% 79.2746%
         entropy 8 9 81.2293% 86.4613% 78.9870%
         entropy 9 3 81.0198% 81.9724% 80.6115%
         entropy 9 4 81.7116% 83.1662% 81.0882%
         entropy 9 5 81.2323% 84.2406% 79.9430%
         entropy 9 6 82.0460% 85.1720% 80.7063%
         entropy 9 7 81.1391% 85.9361% 79.0832%
         entropy 9 8 81.7195% 85.8644% 79.9430%
         entropy 9 9 81.3287% 86.1270% 79.2723%
0.8223999233146568 entropy 8 4


In [196]:
# report the accuracy on test set
# k=10
# begin answer
base_learner = DecisionTree(criterion=best_method, max_depth=best_max_depth, min_samples_leaf=best_min_samples_leaf, sample_feature=True)
rf = RandomForest(base_learner=base_learner, n_estimator=10, seed=2020)
# end answer
rf.fit(X, y)
print("Accuracy on train set: {}".format(accuracy(y, rf.predict(X))))
print("Accuracy on test set: {}".format(accuracy(y_test, rf.predict(X_test))))

Accuracy on train set: 0.8299904489016237
Accuracy on test set: 0.8206106870229007


In [170]:
# TODO: Train the best RandomForest that you can. You should choose some 
# hyper-parameters such as max_depth, and min_samples_in_leaf 
# according to the cross-validation result.

# For k = 100

from sklearn.model_selection import KFold
kf = KFold(n_splits=5, shuffle=True, random_state=2020)

best_acc, best_min_samples_leaf, best_max_depth = 0, 0, 0
best_method = ""
for now_method in ['infogain_ratio', 'gini', 'entropy']:
    for now_min_samples_leaf in range(1, 10, 2):
        for now_max_depth in range(3, 6, 1):
            base_learner = DecisionTree(criterion=now_method, max_depth=now_max_depth, min_samples_leaf=now_min_samples_leaf, sample_feature=True)
            rf = RandomForest(base_learner=base_learner, n_estimator=100, seed=2020)
            ave_valid_acc, ave_train_acc, ave_com_acc = 0, 0, 0
            for train_indice, valid_indice in kf.split(X, y):
                X_train_fold, y_train_fold = X.loc[train_indice], y.loc[train_indice]
                X_val_fold, y_val_fold = X.loc[valid_indice], y.loc[valid_indice]
                rf.fit(X_train_fold, y_train_fold)
                y_train_pred = rf.predict(X_train_fold) 
                y_valid_pred = rf.predict(X_val_fold)
                train_acc = accuracy(y_train_fold, y_train_pred)
                valid_acc = accuracy(y_val_fold, y_valid_pred)
                ave_train_acc += train_acc
                ave_valid_acc += valid_acc
                ave_com_acc += valid_acc * 0.7 + train_acc * 0.3
                
            ave_train_acc /= 5
            ave_valid_acc /= 5
            ave_com_acc /= 5
            print("%16s %d %d %.4f%% %.4f%% %.4f%%" % (now_method, now_min_samples_leaf, now_max_depth, ave_com_acc*100, ave_train_acc*100, ave_valid_acc*100))
            if ave_com_acc > best_acc:
                best_acc = ave_com_acc
                best_method = now_method
                best_min_samples_leaf = now_min_samples_leaf
                best_max_depth = now_max_depth
    
    
# begin answer
print (best_acc, best_method, best_min_samples_leaf, best_max_depth)
# end answer

  infogain_ratio 1 3 79.0561% 79.4415% 78.8909%
  infogain_ratio 1 4 80.3463% 81.0650% 80.0383%
  infogain_ratio 1 5 81.1680% 82.6885% 80.5163%
  infogain_ratio 3 3 79.2239% 79.7756% 78.9875%
  infogain_ratio 3 4 80.4344% 81.1365% 80.1335%
  infogain_ratio 3 5 81.3780% 82.4975% 80.8982%
  infogain_ratio 5 3 78.8731% 79.2740% 78.7013%
  infogain_ratio 5 4 80.3886% 81.2083% 80.0374%
  infogain_ratio 5 5 80.8788% 82.6168% 80.1340%
  infogain_ratio 7 3 79.1757% 79.3937% 79.0823%
  infogain_ratio 7 4 80.7734% 81.3755% 80.5154%
  infogain_ratio 7 5 80.9483% 82.4020% 80.3254%
  infogain_ratio 9 3 78.8872% 79.3221% 78.7008%
  infogain_ratio 9 4 80.2936% 81.1126% 79.9426%
  infogain_ratio 9 5 81.0966% 82.4497% 80.5167%
            gini 1 3 81.6033% 82.1395% 81.3734%
            gini 1 4 81.7639% 83.7870% 80.8968%
            gini 1 5 81.8849% 85.5301% 80.3226%
            gini 3 3 81.7229% 82.0918% 81.5648%
            gini 3 4 82.0078% 83.9302% 81.1839%
            gini 3 5 81.9693% 85.3629% 8

In [200]:
# report the accuracy on test set
# k=100
# begin answer
base_learner = DecisionTree(criterion='gini', max_depth=best_max_depth, min_samples_leaf=best_min_samples_leaf, sample_feature=True)
rf = RandomForest(base_learner=base_learner, n_estimator=100, seed=2020)
# end answer
rf.fit(X, y)
print("Accuracy on train set: {}".format(accuracy(y, rf.predict(X))))
print("Accuracy on test set: {}".format(accuracy(y_test, rf.predict(X_test))))

Accuracy on train set: 0.8347659980897804
Accuracy on test set: 0.8206106870229007


## Adaboost
Please implement the adaboost model in **adaboost.py**. The PDF file provides some hints for this part.

In [28]:
from decision_tree import DecisionTree
from adaboost import Adaboost
import time

t1 = time.time()
base_learner = DecisionTree(criterion='entropy', max_depth=1, min_samples_leaf=1, sample_feature=False)
ada = Adaboost(base_learner=base_learner, n_estimator=100, seed=2020)
ada.fit(X, y)

y_train_pred = ada.predict(X)

print("Accuracy on train set: {}".format(accuracy(y, y_train_pred)))
print("Accuracy on test set: {}".format(accuracy(y_test, ada.predict(X_test))))

t2 = time.time()

print ("Time:", t2-t1)

Accuracy on train set: 0.8089780324737345
Accuracy on test set: 0.8129770992366412
Time: 41.196943283081055


In [30]:
# TODO: Train the best Adaboost that you can. You should choose some 
# hyper-parameters such as max_depth, and min_samples_in_leaf 
# according to the cross-validation result.
# begin answer

from sklearn.model_selection import KFold
kf = KFold(n_splits=5, shuffle=True, random_state=2020)

best_acc, best_min_samples_leaf, best_max_depth = 0, 0, 0
best_method = ""
for now_method in ['infogain_ratio', 'gini', 'entropy']:
    for now_min_samples_leaf in range(1, 6, 2):
        for now_max_depth in range(1, 4, 1):
            base_learner = DecisionTree(criterion=now_method, max_depth=now_max_depth, min_samples_leaf=now_min_samples_leaf, sample_feature=False)
            ada = Adaboost(base_learner=base_learner, n_estimator=50, seed=2020)
            ave_valid_acc, ave_train_acc, ave_com_acc = 0, 0, 0
            for train_indice, valid_indice in kf.split(X, y):
                X_train_fold, y_train_fold = X.loc[train_indice], y.loc[train_indice]
                X_val_fold, y_val_fold = X.loc[valid_indice], y.loc[valid_indice]
                ada.fit(X_train_fold, y_train_fold)
                y_train_pred = ada.predict(X_train_fold) 
                y_valid_pred = ada.predict(X_val_fold)
                train_acc = accuracy(y_train_fold, y_train_pred)
                valid_acc = accuracy(y_val_fold, y_valid_pred)
                ave_train_acc += train_acc
                ave_valid_acc += valid_acc
                ave_com_acc += valid_acc * 0.7 + train_acc * 0.3
                
            ave_train_acc /= 5
            ave_valid_acc /= 5
            ave_com_acc /= 5
            print("%16s %d %d %.4f%% %.4f%% %.4f%%" % (now_method, now_min_samples_leaf, now_max_depth, ave_com_acc*100, ave_train_acc*100, ave_valid_acc*100))
            if ave_com_acc > best_acc:
                best_acc = ave_com_acc
                best_method = now_method
                best_min_samples_leaf = now_min_samples_leaf
                best_max_depth = now_max_depth
    
    
# begin answer
print (best_acc, best_method, best_min_samples_leaf, best_max_depth)
# end answer
# end answer

  infogain_ratio 1 1 80.9768% 81.1604% 80.8982%
  infogain_ratio 1 2 80.5538% 84.4318% 78.8918%
  infogain_ratio 1 3 80.6880% 87.5596% 77.7430%
  infogain_ratio 3 1 80.9768% 81.1604% 80.8982%
  infogain_ratio 3 2 80.4868% 84.4318% 78.7961%
  infogain_ratio 3 3 80.3455% 87.5357% 77.2641%
  infogain_ratio 5 1 80.9768% 81.1604% 80.8982%
  infogain_ratio 5 2 80.1599% 84.2407% 78.4110%
  infogain_ratio 5 3 80.4390% 87.8461% 77.2645%
            gini 1 1 80.6669% 81.0173% 80.5167%
            gini 1 2 81.0644% 85.2438% 79.2732%
            gini 1 3 79.6206% 88.0134% 76.0237%
            gini 3 1 80.6669% 81.0173% 80.5167%
            gini 3 2 81.0640% 85.2438% 79.2727%
            gini 3 3 79.6530% 87.8939% 76.1212%
            gini 5 1 80.6669% 81.0173% 80.5167%
            gini 5 2 81.1712% 84.9331% 79.5589%
            gini 5 3 80.3509% 87.9894% 77.0772%
         entropy 1 1 80.5501% 80.8501% 80.4215%
         entropy 1 2 81.4017% 85.0284% 79.8473%
         entropy 1 3 79.8755% 87.9655% 7

In [35]:
# report the accuracy on test set
# begin answer
# end answer
base_learner = DecisionTree(criterion='infogain_ratio', max_depth=1, min_samples_leaf=3, sample_feature=False)
ada = Adaboost(base_learner=base_learner, n_estimator=50, seed=2020)

ada.fit(X, y)
print("Accuracy on train set: {}".format(accuracy(y, ada.predict(X))))
print("Accuracy on test set: {}".format(accuracy(y_test, ada.predict(X_test))))

Accuracy on train set: 0.8118433619866284
Accuracy on test set: 0.8091603053435115


In [36]:
import sys

train_x = np.array(X)
train_y = np.array(y)
test_x = np.array(X_test)
test_y = np.array(y_test)

In [67]:
sys.path.append("../../assignment1/bayes_decision_rule")
from posterior import posterior
# Some modification must be done if use my own naive bayes, so I use sklearn.

from sklearn.naive_bayes import GaussianNB

clf = GaussianNB()
clf.fit(train_x, train_y)

print("naive bayes train accuracy:", accuracy(clf.predict(train_x), train_y))
print("naive bayes test accuracy:", accuracy(clf.predict(test_x), test_y))

naive bayes train accuracy: 0.766953199617956
naive bayes test accuracy: 0.7557251908396947


In [70]:
from sklearn.linear_model import Perceptron

clf = Perceptron(shuffle=False)
print (clf.fit(train_x, train_y))

print("perceptron train accuracy:", accuracy(clf.predict(train_x), train_y))
print("perceptron test accuracy:", accuracy(clf.predict(test_x), test_y))

Perceptron(alpha=0.0001, class_weight=None, early_stopping=False, eta0=1.0,
           fit_intercept=True, max_iter=1000, n_iter_no_change=5, n_jobs=None,
           penalty=None, random_state=0, shuffle=False, tol=0.001,
           validation_fraction=0.1, verbose=0, warm_start=False)
perceptron train accuracy: 0.6227316141356256
perceptron test accuracy: 0.6183206106870229


In [71]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression()
print (clf.fit(train_x, train_y))

print("perceptron train accuracy:", accuracy(clf.predict(train_x), train_y))
print("perceptron test accuracy:", accuracy(clf.predict(test_x), test_y))

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)
perceptron train accuracy: 0.7975167144221585
perceptron test accuracy: 0.7862595419847328


In [75]:
from sklearn import svm

clf = svm.SVC()
print (clf.fit(train_x, train_y))

print("perceptron train accuracy:", accuracy(clf.predict(train_x), train_y))
print("perceptron test accuracy:", accuracy(clf.predict(test_x), test_y))

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
perceptron train accuracy: 0.8185291308500477
perceptron test accuracy: 0.8129770992366412


In [81]:
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier()
print (clf.fit(train_x, train_y))

print("perceptron train accuracy:", accuracy(clf.predict(train_x), train_y))
print("perceptron test accuracy:", accuracy(clf.predict(test_x), test_y))

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100,), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=200,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)
perceptron train accuracy: 0.8175740210124164
perceptron test accuracy: 0.8282442748091603


d:\python37\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [76]:
from sklearn.neighbors import KNeighborsClassifier

clf = KNeighborsClassifier(n_neighbors=2)
print (clf.fit(train_x, train_y))

print("perceptron train accuracy:", accuracy(clf.predict(train_x), train_y))
print("perceptron test accuracy:", accuracy(clf.predict(test_x), test_y))

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=2, p=2,
                     weights='uniform')
perceptron train accuracy: 0.828080229226361
perceptron test accuracy: 0.7786259541984732


In [77]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier()
print (clf.fit(train_x, train_y))

print("perceptron train accuracy:", accuracy(clf.predict(train_x), train_y))
print("perceptron test accuracy:", accuracy(clf.predict(test_x), test_y))

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')
perceptron train accuracy: 0.8767908309455588
perceptron test accuracy: 0.8015267175572519


In [79]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()
print (clf.fit(train_x, train_y))

print("perceptron train accuracy:", accuracy(clf.predict(train_x), train_y))
print("perceptron test accuracy:", accuracy(clf.predict(test_x), test_y))

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)
perceptron train accuracy: 0.8767908309455588
perceptron test accuracy: 0.8129770992366412


In [80]:
from sklearn.ensemble import AdaBoostClassifier

clf = AdaBoostClassifier()
print (clf.fit(train_x, train_y))

print("perceptron train accuracy:", accuracy(clf.predict(train_x), train_y))
print("perceptron test accuracy:", accuracy(clf.predict(test_x), test_y))

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1.0,
                   n_estimators=50, random_state=None)
perceptron train accuracy: 0.8099331423113658
perceptron test accuracy: 0.8129770992366412


In [162]:
from GBDT import GBDT
from binary_decision_tree import BinaryDecisionTree

base_learner = BinaryDecisionTree(max_depth = 5, min_samples_leaf = 3)
gbdt = GBDT(base_learner = base_learner, n_estimator = 100, learning_rate = 0.6)

gbdt.fit(X, y)
print (accuracy(gbdt.predict(X),y))
print (accuracy(gbdt.predict(X_test),y_test))

0.8213944603629417
0.8091603053435115


In [230]:
# prune

from decision_tree import DecisionTree
import copy
import time


X_train, X_valid = X[:670], X[670:]
y_train, y_valid = y[:670], y[670:]

t1 = time.time()
ave_train, ave_test, ave_node = 0, 0, 0

for repeat in range(10):
    dt = DecisionTree(criterion='entropy', max_depth=10, min_samples_leaf=repeat, sample_feature=False, last_cut = True)
    #dt.fit(X, y)
    dt.fit_with_cut(X_train, y_train, X_valid, y_valid)
    ave_train += accuracy(y, dt.predict(X))
    ave_test += accuracy(y_test, dt.predict(X_test))
    ave_node += dt.calc_nodes(dt._tree)
    
ave_train /= 10
ave_test /= 10
ave_node /= 10

t2 = time.time()
print ("time: ", t2-t1)
print("Average accuracy on train set: {}".format(ave_train))
print("Average accuracy on test set: {}".format(ave_test))
print("Average nodes: {}".format(ave_node))

time:  2.9122109413146973
Average accuracy on train set: 0.8187201528175742
Average accuracy on test set: 0.8076335877862595
Average nodes: 44.4


In [263]:
# continous

from GBDT import GBDT
from binary_decision_tree import BinaryDecisionTree

base_learner = BinaryDecisionTree(max_depth = 5, min_samples_leaf = 3)
gbdt = GBDT(base_learner = base_learner, n_estimator = 100, learning_rate = 0.6)

gbdt.fit(X, y)
pred_test = gbdt.predict(X_test)
print (accuracy(gbdt.predict(X),y))
print (accuracy(pred_test,y_test))


0.8213944603629417
0.8091603053435115


In [272]:
#pred_test, real_test = gbdt.predict(X_test)

def choose(th):
    chosen_data = np.logical_or(real_test <= th, real_test >= 1 - th)
    print (np.sum(chosen_data), accuracy(pred_test[chosen_data], y_test[chosen_data]))

choose(0.5)
choose(0.3)
choose(0.1)


262 0.8091603053435115
200 0.865
72 0.9027777777777778


In [261]:
from random_forest import RandomForest

import time

import pickle
pickle.dump(X, file = open("X", "wb"))
pickle.dump(y, file = open("y", "wb"))

print (time.time())
base_learner = DecisionTree(criterion='entropy', max_depth=5, min_samples_leaf=5, sample_feature=True)
rf = RandomForest(base_learner=base_learner, n_estimator=10, seed=2020, is_parallel = True)

rf.fit(X[0:300],y[0:300])
print (time.time())
y_train_pred = rf.predict(X[0:300])
print (time.time())

#print("Accuracy on train set: {}".format(accuracy(y, y_train_pred)))
#print("Accuracy on test set: {}".format(accuracy(y_test, rf.predict(X_test))))



1593353846.7905393
1593353846.949146
True
Start


KeyboardInterrupt: 